<a href="https://colab.research.google.com/github/Sussex123456143/Chatbot/blob/main/Animal_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#import libraries
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
from sklearn.utils import class_weight



In [ ]:
def preprocess(image,label):
  return tf.image.resize(image,(64,64))/255.0 ,label

train_data, test_data = tfds.load("cats_vs_dogs", split=["train[:80%]", "train[80%:]"], as_supervised=True)
train_data = train_data.map(preprocess).batch(32).prefetch(tf.data.AUTOTUNE)
test_data = test_data.map(preprocess).batch(32).prefetch(tf.data.AUTOTUNE)

In [ ]:
model = tf.keras.Sequential([

    # The 16 filters (3 x 3) scan the input image, each detecting different features (ex: edges)
    # ReLU (Rectified Linear Unit) is an activation function that makes sure the layer focuses on important features
    # The model takes 64 x 64 pixel images with 3 color channels (RGB)
    tf.keras.layers.Conv2D(16, (3, 3), activation='relu', input_shape=(64, 64, 3)),

    # takes feature map created by Conv2D and shrinks it by using a 2 x 2 matrix and retains important information from each matrix
    tf.keras.layers.MaxPooling2D(2, 2),

    # looks for patterns/features from previous layer
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),

    # shrinks feature map amde by previous layer and retains important info
    tf.keras.layers.MaxPooling2D(2, 2),

    # averages values for every channel (RGB) in map
    tf.keras.layers.GlobalAveragePooling2D(),

    # Classifier layer where 2 shows number of classes to indentify (cat or dog)
    # softmax is a activation function that converts the scores generated by previous layer into probability between [0, 1]
    tf.keras.layers.Dense(2, activation='softmax')
])


model.compile(optimizer="adam",loss="sparse_categorical_crossentropy",metrics=["accuracy"])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
labels = np.concatenate([y for x, y in train_data],axis=0)
class_weights = dict(enumerate(class_weight.compute_class_weight('balanced', classes=np.unique(labels), y=labels)))


In [ ]:
model.fit(train_data,validation_data = test_data, epochs = 2, class_weight = class_weights)


Epoch 1/2
582/582 ━━━━━━━━━━━━━━━━━━━━ 86s 144ms/step - accuracy: 0.5545 - loss: 0.6812 - val_accuracy: 0.6111 - val_loss: 0.6526
Epoch 2/2
582/582 ━━━━━━━━━━━━━━━━━━━━ 86s 148ms/step - accuracy: 0.6180 - loss: 0.6496 - val_accuracy: 0.6320 - val_loss: 0.6418


In [ ]:
from tensorflow.keras.preprocessing import image
image_path = ""
img = image.load_img(image_path,target_size = (64,64))
img_array = np.expand_dims(image.img_t_array(img),axis=0)/255.0
predition = model.predict(img_array)
label = np.argmax(prediction)
if (label == 0):
  print("Dog")
else:
  print("Cat")